In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import predictor

from data.imdb import Tokenizer

In [2]:
tf.__version__

'1.12.0'

In [3]:
export_dir = "models/imdb_sentiment/export/model/1552107126"
seq_len = 256
vocab_size = 2048

In [4]:
p = predictor.from_saved_model(
    export_dir, 
    input_names={"input_text": "input_text:0"}, 
    output_names={
        "prediction": "dense/Softmax:0",
        "attention_0": "attention/attention_weights:0",
        "attention_1": "attention_1/attention_weights:0",
    },
)

INFO:tensorflow:Restoring parameters from models/imdb_sentiment/export/model/1552107126/variables/variables


I0309 13:35:01.246718 4596434368 tf_logging.py:115] Restoring parameters from models/imdb_sentiment/export/model/1552107126/variables/variables


In [5]:
tok = Tokenizer(vocab_size, seq_len)

In [6]:
tok.load_index("models/imdb_sentiment/data/word_index.json")

In [7]:
inputs = list(tok.transform([
    b"this movie is excellent", 
    b"this movie is good", 
    b"this movie sucks",
    b"That movie was absolutely awful",
    b"The acting was a bit lacking",
    b"The film was creative and surprising",
    b"Absolutely fantastic!"
]))

In [8]:
outputs = p({"input_text": inputs})
outputs["prediction"]

array([[1.4082224e-13, 1.0000000e+00],
       [5.4077776e-10, 1.0000000e+00],
       [1.0000000e+00, 8.3034447e-19],
       [1.0000000e+00, 1.5380015e-13],
       [5.9238344e-01, 4.0761656e-01],
       [2.6104544e-11, 1.0000000e+00],
       [3.5190692e-10, 1.0000000e+00]], dtype=float32)